# Sample file to generate partition on an articles using Community Detection
* Input - article text
* Output - partition assignment
* author: harin

In [ ]:
import pandas as pd
import numpy as np
import spacy

In [2]:
file = open('../data/raw/2018_07_19_04_59_08/articles.txt', 'r', encoding = 'utf8')

In [8]:
pubId, canonicalUrl,firstScrape,title,text,lang_reliability = [],[],[],[],[],[]
for article in articles_dt:    
    row = article.split('\t')
    pubId.append(row[0])
    canonicalUrl.append(row[4])
    firstScrape.append(row[5])
    lang_reliability.append(row[7])
    title.append(row[8])
    text.append(row[9])
    
articles_df = pd.DataFrame()
articles_df['pubId'] = pubId
articles_df['canonicalUrl'] = canonicalUrl
articles_df['firstScrape'] = firstScrape
articles_df['title'] = title
articles_df['text'] = text

In [9]:
articles_df.shape

(0, 5)

In [6]:
articles_df.canonicalUrl.unique().shape

(213605,)

In [5]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity

SAMPLE_SIZE=10000

nlp = spacy.load("en_core_web_sm")
sample_df = articles_df.sample(n=SAMPLE_SIZE)
docs = list(nlp.pipe(sample_df.text))
embs = [doc.vector for doc in docs]

embs_df = pd.DataFrame(embs)
embs_df = embs_df.dropna()
cs = cosine_similarity(embs_df)

# Construct graph

In [6]:
cs

array([[1.        , 0.91510321, 0.69478763, ..., 0.95567983, 0.88125627,
        0.75622025],
       [0.91510321, 1.        , 0.86309751, ..., 0.91872582, 0.81169764,
        0.89070246],
       [0.69478763, 0.86309751, 1.        , ..., 0.73390102, 0.60940438,
        0.91132334],
       ...,
       [0.95567983, 0.91872582, 0.73390102, ..., 1.        , 0.90681523,
        0.74009412],
       [0.88125627, 0.81169764, 0.60940438, ..., 0.90681523, 1.        ,
        0.62462789],
       [0.75622025, 0.89070246, 0.91132334, ..., 0.74009412, 0.62462789,
        1.        ]])

In [7]:
import networkx as nx
import community
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
G = nx.Graph()

In [9]:
threshold = 0.5

In [11]:
for i in range(len(cs)):
    similarities = cs[i]
    for j, sim in enumerate(similarities):
        if i == j: continue
        if sim > threshold:
            G.add_edge(i, j)

In [ ]:
pos = nx.spring_layout(G)
nx.draw_networkx_edges(G, pos, alpha=0.5)

In [ ]:
#first compute the best partition
partition = community.best_partition(G)

#drawing
size = float(len(set(partition.values())))
pos = nx.spring_layout(G)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))

nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

In [ ]:
num_cluster = max([v for k,v in partition.items()]) + 1

In [ ]:
num_cluster